# Momi2 analysis
## Notebook must be run in python3

In [2]:
import momi

In [23]:
popfile = "simpops.txt"

dat = !head tmp.vcf
## Relying on the fact that the vcf files 5th line contains the samples
## which are indexed as 9 and on
samps = dat[5].split()[9:]
print(samps)
samps_per_pop = int(len(samps)/2)
popdict = {x:"pop1" for x in samps[:samps_per_pop]}
popdict.update({x:"pop2" for x in samps[samps_per_pop:]})
print(popdict)
with open(popfile, 'w') as outfile:
    for k,v in popdict.items():
        outfile.write("{}\t{}\n".format(k,v))
!cat simpops.txt

['msp_0', 'msp_1', 'msp_2', 'msp_3', 'msp_4', 'msp_5', 'msp_6', 'msp_7']
{'msp_0': 'pop1', 'msp_1': 'pop1', 'msp_2': 'pop1', 'msp_3': 'pop1', 'msp_4': 'pop2', 'msp_5': 'pop2', 'msp_6': 'pop2', 'msp_7': 'pop2'}
msp_0	pop1
msp_1	pop1
msp_2	pop1
msp_3	pop1
msp_4	pop2
msp_5	pop2
msp_6	pop2
msp_7	pop2


In [69]:
!bgzip -c tmp.vcf > tmp.vcf.gz
!tabix tmp.vcf.gz

In [70]:
def vcf2bed(vcfinput, bedoutput):
    lines = open(vcfinput).readlines()
    snps = [l.split()[:2] for l in lines if not l.startswith("#")]
    with open(bedoutput, 'w') as outfile:
        for snp in snps:
            outfile.write("{}\t{}\t{}\n".format(snp[0], snp[1], int(snp[1])+1))
vcf2bed("tmp.vcf", "tmp.bed")
!head tmp.bed

1	13	14
1	131	132
2	69	70
2	111	112
3	47	48
4	46	47
4	66	67
4	99	100
4	109	110
5	106	107


In [71]:
%%bash
python -m momi.read_vcf --no_aa --verbose tmp.vcf.gz simpops.txt tmp_allele_counts.gz --bed tmp.bed
gunzip -c tmp_allele_counts.gz | head

{
	"populations": ["pop1", "pop2"],
	"use_folded_sfs": true,
	"length": 3130,
	"n_read_snps": 99,
	"n_excluded_snps": 0,
	"configs": [
		[[8, 0], [7, 1]],
		[[8, 0], [6, 2]],
		[[8, 0], [5, 3]],


/home/isaac/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [88]:
!python -m momi.extract_sfs tmp_sfs.gz 10 tmp_allele_counts.gz
sfs = momi.Sfs.load("tmp_sfs.gz")
print("Avg pairwise heterozygosity", sfs.avg_pairwise_hets[:5])
print("populations", sfs.populations)
print("percent missing data per population", sfs.p_missing)

/home/isaac/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Avg pairwise heterozygosity [[1.57142857 2.28571429]
 [1.25       1.17857143]
 [1.67857143 1.5       ]
 [2.07142857 0.75      ]
 [3.32142857 1.5       ]]
populations ('pop1', 'pop2')
percent missing data per population [0. 0.]


In [111]:
no_migration_model = momi.DemographicModel(N_e=1e5,
                                           muts_per_gen=1e-8,
                                           gen_time=1)

no_migration_model.set_data(sfs, length=1e5)

no_migration_model.add_time_param("tdiv")

no_migration_model.add_leaf("pop1")
no_migration_model.add_leaf("pop2")
no_migration_model.move_lineages("pop1", "pop2", t="tdiv")

no_migration_model.optimize()

            fun: 3.4423825612194747
            jac: array([-2.46314709e-12])
  kl_divergence: 3.4423825612194747
 log_likelihood: -586.0527145224539
        message: 'Local minimum reached (|pg| ~= 0)'
           nfev: 10
            nit: 4
     parameters: ParamsDict({'tdiv': 225895.46294887832})
         status: 0
        success: True
              x: array([225895.46294888])

In [57]:
!python -m momi.extract_sfs

/home/isaac/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
usage: extract_sfs.py [-h] [--verbose] out n_blocks files [files ...]
extract_sfs.py: error: the following arguments are required: out, n_blocks, files
